## CartPole
### Reinforcement Learning (DQN) Tutorial text
Link: https://h-huang.github.io/tutorials/intermediate/reinforcement_q_learning.html

#### Imports

In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch as pt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T



env = gym.make('CartPole-v0').unwrapped

# Set up matplotlib
if is_ipython := 'inline' in matplotlib.get_backend():
    from IPython import display

plt.ion()

# If GPU is to be used
device = pt.device('cuda' if pt.cuda.is_available() else 'cpu')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  logger.warn(


#### Replay Memory

experience replay memory for training our DQN. It stores the transitions that the agent observes, which can be reused later. By sampeling from it randomly the transitions that buld up a batch are decorrelated. It has been shown that this greatly stabilizes and improves the training procedure.

Two classes in use:
* Transition - a named tuple represented by a single transition in our environment. It maps (state, action) to this paired (next_state, reward) result
* ReplayMemory - a cyclic buffer of bounded size that holds the transitions observed recently. It also implements a .sample() method which can be used to select a random batch of transitions for training.

In [2]:
Transition = namedtuple('Transition', 
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    '''ReplayMemory stores the transitions that the agent observes, allowing us to reuse this data later. 
    
    By sampling from it randomly, the transitions that build up a batch are decorrelated.'''
    def __init__(self, capacity) -> None:
        self.memory = deque([], maxlen=capacity)
    
    def push(self, *args):
        '''Saves a transition'''
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

### DQN algorithm

Our environment is deterministic, so all equations presented here are also formulated deterministically for the sake of simplicity. In the reinforcement learning literature, they would also contain expectations over stochastic transitions in the environment (randomness with a pattern).

Our aim will be to train a policy that tries to maximize the discounted, cumulative reward 
R_t0 = sum(discountRate * r_t)

The main idea behind Q-learning is that if we have a function Q*: State*Action -> R, That couldtell us what our return would be if we were to take an action in a given state, then we could easily construct a policy that maximizes or rewards:

pi*(s) = argmax Q*(s,a)

However, we don't know everything in the world, so we dont have access to Q*, but we can approximate this. Therefore our updated training rule we can use this Q function

Q^pi(s,a) = r+discount*Q^pi(s', pi(s'))

To minimize the error we can use Huber Loss: https://en.wikipedia.org/wiki/Huber_loss
Which acts like the mean sqeared error, when the error is small, but like the mean absolute error when its large making this method robust:

Q-net
Our model will be feed forward neural network that takes in the difference between the current and previous screen patches. It has two outputs, representing Q(s, left) or Q(s, right), where right or left is the action and s is the input to the network. 



In [3]:
class DQN(nn.Module):
    def __init__(self, input_units:int, output_units:int, hidden_units:int=128) -> None:
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(input_units, hidden_units)
        self.layer2 = nn.Linear(hidden_units, hidden_units)
        self.layer3 = nn.Linear(hidden_units, output_units)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)



### Training

#### Hyperparams and utilities
This cell instantiates our model and its optimizer and defines some utilities:

* select_action - selects action according to epsilon greedy policy. The probability of choosing a random action will start at EPS_Start and move towards EPS_END where EPS_DECAY is the rate of decay. This creates a gradual transformation from exploration to exploitation
* plot_durations - a helper for plotting the duration of episodes, along with the averate for the last 100 episodes (The measure used in the official evaluations). The plot underneath the cell containing the main training loop and will update every episode.



In [10]:
# Params
# BATCH_SIZE is the number of transitions sampled from the replay buffer
BATCH_SIZE = 128

# GAMMA is the discount factor as mentioned in the previous section
GAMMA = 0.99

# EPS_START is the starting value of epsilon
EPS_START = 0.9

# EPS_END is the final value of epsilon
EPS_END = 0.05

# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
EPS_DECAY = 1000

# TAU is the update rate of the target network
TAU = 0.005

# LR is the learning rate of the ``AdamW`` optimizer
LR = 1e-4

# ------------------------

# Get num of actions from gym action space
n_actions = env.action_space.n # left or right
# Get the number of state observations
state = env.reset()
n_observations = len(state)

# A nn which takes an observation of env as input and outputs a probability over possible actions
policy_net = DQN(n_observations, n_actions).to(device)

# the target_net is a copy of the policy net which stabilizes the learning process to ensure that no destructive changes are made to the model
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)

steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with pt.no_grad():
            # basically return the largest column in each row.
            # Second column on max result is index of where max element was found
            # So we pick action with the largest expected reward.
            return policy_net(state).max(1).indices.view(1,1)
    else:
        # returns a random sample
        return pt.tensor([[env.action_space.sample()]], device=device, dtype=pt.long)

# an episode is basically an entire step through the learning process
episode_durations = []

# continue with plot_durations